# asyncio私房手册

目录和官方文档一致，主要补充学习官方文档中的一些心得和细节。
- [官方文档](https://docs.python.org/zh-cn/3/library/asyncio.html)
- [Guide to Concurrency in Python with Asyncio](https://www.integralist.co.uk/posts/python-asyncio/)

## 高级API

### task

`create_task`运行的时间非常微妙，先看代码：
```python
async def func1():
    print("func1 start!", time.strftime("%X"))
    await asyncio.sleep(1)
    print("func1 end!", time.strftime("%X"))


async def func2():
    print("func2 start!", time.strftime("%X"))
    await asyncio.sleep(2)
    print("func2 end!", time.strftime("%X"))


async def main():
    task1 = asyncio.create_task(func1())  # 任务被放入事件循环，但未运行
    task2 = asyncio.create_task(func2())  # 任务被放入事件循环，但未运行
    print("main start!")
    await task1  # 主程序挂起，开始事件循环，执行任务
    await task2
    print("main end!")


asyncio.run(main())
```
输出为：
```
main start!
func1 start! 09:07:25
func2 start! 09:07:25
func1 end! 09:07:26
func2 end! 09:07:27
main end!
```
可见，协程并非是在`create_task`的时候就开始运行。task运行的时间非常微妙，`create_task`创建了一个任务，并将其放入后台的事件循环队列中，然后马上回到主程序，但是此时并没有运行，直到遇到紧接着的`await`，即主程序被挂起，开始执行事件循环里的事件，此时任务开始运行。

把`main`改一改，可以看得更清晰。
```python
async def main():
    task1 = asyncio.create_task(func1())
    task2 = asyncio.create_task(func2())
    await asyncio.sleep(.1)
    print("main start!")
    print("main end!")
```
输出为：
```
func1 start! 09:29:57
func2 start! 09:29:57
main start!
main end!
```
可见，即使没有`await task1`，任务也会运行，因为任务前面已经被加入到事件循环队列，当遇到`await asyncio.sleep`时，主程序挂起，运行事件循环里的任务，遇到任务中的`await`时，又回到主程序。此时没有`await task1`这样的语句来获取任务结果，因此直接就结束了。

### gather

gather可以将协程包装成任务加入事件循环队列，也就是使用gather，可以直接传入协程而不需要先创建任务，gather的异常处理有点微妙：
```python
async def func1():
    print("func1 start!", time.strftime("%X"))
    raise RuntimeError("func1 failed!")
    await asyncio.sleep(1)
    print("func1 end!", time.strftime("%X"))
    return "func1"


async def func2():
    print("func2 start!", time.strftime("%X"))
    await asyncio.sleep(2)
    print("func2 end!", time.strftime("%X"))
    return "func2"


async def main():
    print("main start!", time.strftime("%X"))
    try:
        results = await asyncio.gather(func1(), func2())  # 异常抛出
        print("results is", results)
    except RuntimeError as e:
        print(e)
        await asyncio.sleep(3)  # 如果继续等待，则可以看到func2仍然运行了
    print("main end!", time.strftime("%X"))


asyncio.run(main())
```
输出为：
```
main start! 10:01:01
func1 start! 10:01:01
func2 start! 10:01:01
func1 failed!
func2 end! 10:01:03
main end! 10:01:04
```
默认情况下，如果任务队列里的任务抛出错误，会向上传播，要注意的是，队列里的其它任务不会取消还是会继续运行，如果设置`return_exception=True`，则异常不会抛出，会和结果一起返回。如：
```python
async def main():
    print("main start!", time.strftime("%X"))
    results = await asyncio.gather(func1(), func2(), return_exceptions=True)
    print("results is", results)
    print("main end!", time.strftime("%X"))
```
此时返回的结果是：
```python
main start! 10:08:52
func1 start! 10:08:52
func2 start! 10:08:52
func2 end! 10:08:54
results is [RuntimeError('func1 failed!'), 'func2']
main end! 10:08:54
```
gather返回一个future，可以在这个future上调用cancel，但是它的一些细节还是没有弄清楚，以下记录了stack overflow的一些问题：
- [How to cancel all remaining tasks in gather if one fails?](https://stackoverflow.com/questions/59073556/how-to-cancel-all-remaining-tasks-in-gather-if-one-fails)

### wait

`wait`和`gather`有点类似，都是获取任务结果的方法，不过`wait`比`gather`更灵活：
```python
async def main():
    print("main start!", time.strftime("%X"))
    done, pending = await asyncio.wait([func1(), func2()], timeout=2)
    print(done.pop().result())
    for p in pending:
        p.cancel()
    print("main end!", time.strftime("%X"))
```
输出为：
```
main start! 14:59:21
func2 start! 14:59:21
func1 start! 14:59:21
func1 end! 14:59:22
func1
main end! 14:59:23
```
`wait`返回done和pending两个集合，分别包含已经完成的任务和被挂起的任务，可以通过`timeout`关键字设定等待时间，也可以通过`return_when`关键字设定返回的条件，比如`asyncio.FIRST_COMPLETED`，表示第一个就返回。

### as_completed

也是用来获取结果，感觉和`gather`差不多，不过返回一个生成器，然后迭代生成器来获取结果，注意，其返回的结果是`Future`，并且必须调用`await`来等待`Future`的结果，否则会抛出错误：
```python
async def main():
    print("main start!", time.strftime("%X"))
    tasks = asyncio.as_completed([func1(), func2()])
    for task in tasks:
        print(await task)
    print("main end!", time.strftime("%X"))
```

### shield

shield不太好理解，它并不是真正的禁止任务被取消，可以把它理解为任务的一个状态，当使用`shield`包装一个协程，则会把这个协程包装为一个Future并且立即将其放入事件循环队列，当调用`Future.cancel()`方法的时候，`Future`会被标记成“取消”的状态，当`await`这个`Future`的时候，会立即抛出`CancelledError`的错误，但是实际上，这个Future依然在队列中，并未受影响：
```python
async def main():
    print("main start!", time.strftime("%X"))
    task1 = asyncio.create_task(func1())
    task1_with_shield = asyncio.shield(task1) 
    try:
        task1_with_shield.cancel()
        await task1_with_shield  # 只要await task1_with_shield就会抛出CancelledError错误
    except asyncio.CancelledError:
        print("stask1 canceled!")
        await asyncio.sleep(2)  # 实际的task1仍然会运行，不受影响
    print("main end!", time.strftime("%X"))
```
输出为：
```
main start! 14:18:21
stask1 canceled!
func1 start! 14:18:21
func1 end! 14:18:22
main end! 14:18:23
```
因此，可以在捕获了错误以后，仍然获取任务的值：
```python
async def main():
    print("main start!", time.strftime("%X"))
    task1 = asyncio.create_task(func1())
    task1_with_shield = asyncio.shield(task1)
    try:
        task1_with_shield.cancel()
        await task1_with_shield
    except asyncio.CancelledError:
        print("stask1 canceled!")
        r = await task1  # task1仍然在队列中运行，可以获取task1的返回值
        print(r)
    print("main end!", time.strftime("%X"))
```
输出为：
```python
main start! 14:30:42
stask1 canceled!
func1 start! 14:30:42
func1 end! 14:30:43
func1
main end! 14:30:43
```
要注意的是，`shield`如果包装协程的话，不能像上面这样获取协程的值，比如：
```python
async def main():
    print("main start!", time.strftime("%X"))
    task1_with_shield = asyncio.shield(func1())
    try:
        task1_with_shield.cancel()
        await task1_with_shield
    except asyncio.CancelledError:
        print("stask1 canceled!")
        r = await func1()  # 这里的func1()创建了一个新的协程，和task1_with_shield并非队列里的同一个任务
        print(r)
    print("main end!", time.strftime("%X"))
```
输出为：
```
main start! 14:33:15
stask1 canceled!
func1 start! 14:33:15
func1 start! 14:33:15
func1 end! 14:33:16
func1
main end! 14:33:16
func1 end! 14:33:16
```
可见，func1执行了两次，这并不是想要的结果。

### asyncio.run

`asyncio.run`基本上可以近似为以下的代码：
```python
import asyncio, sys, types

def run(coro):
    if sys.version_info >= (3, 7):
        return asyncio.run(coro)

    # Emulate asyncio.run() on older versions

    # asyncio.run() requires a coroutine, so require it here as well
    if not isinstance(coro, types.CoroutineType):
        raise TypeError("run() requires a coroutine object")

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        return loop.run_until_complete(coro)
    finally:
        loop.close()
        asyncio.set_event_loop(None)
```
因此，在使用它的时候，和直接通过loop调用有微妙的区别，来源于以下的代码，在屏幕上运行两个协程，一个会动态的显示`thinking!`，`thinking!`字符前面会有线条转动的动画，一个在2秒钟以后打印结果：
```python
import asyncio
import itertools
import sys


async def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle("|/-\\"):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            await asyncio.sleep(.1)  # 调用spin.cancel的话，异常会在这里抛出
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))


async def slow_function():
    await asyncio.sleep(2)
    return 42


async def supervisor():
    spinner = asyncio.create_task(spin('thinking!'))
    result = await slow_function()
    spinner.cancel()
    return result


def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print(result)


main() 
```
最终输出为42，符合预期，而如果稍微修改一下main函数，使用`asyncio.run`运行，如下：
```python
async def main():
    result = await supervisor()
    print(result)

asyncio.run(main())
```
此时，最终输出为`42thinking!`，问题出在哪里呢？因为两种方式`loop.close()`执行的时间点不同，第一种是先调用`loop.close`，再打印result，而第二种是先打印result，然后再调用`loop.close`。在`supervisor`中，在`spinner.cancel`处，并不会挂起，而是将spinner的状态设置为`cancel`，并返回`result`。当调用`loop.close`时，此时`spin`还在事件循环队列中，处于挂起状态，所以会继续执行`spin`。
1. 第一种情况，相当于先`loop.close`，结束`spin`协程以后，再`print(result)`，相当于会先执行`write(' ' * len(status) + '\x08' * len(status))`，将屏幕上的`thinking!`字符消除，最后打印结果。
2. 第二种情况，相当于先打印result，然后再`loop.close`，执行`write(' ' * len(status) + '\x08' * len(status))`，而由于`print(result)`导致了换行，所以最终结果为`42thinking!`，要解决的话也很简单，可以将`print(result)`改为`print(result, end='')`就行了。

## 低级API

### Future

Future的概念并不是很好理解，它是一个低层级的API，做个类比，可以把它想象成一个放礼物的盒子，现在我们要送礼，但是事先并不知道送什么，我们可以先对这个盒子做一些装饰，然后在未来的某个时刻，再把礼物放进去，最终的目的是获取这个盒子里的礼物。在代码中，我们可以先创建一个`Future`，可以先为这个future添加回调函数（装饰盒子），再在后期设置future的结果（把礼物放进盒子），最终的目的是获取这个结果（得到礼物），比如：
```python
import asyncio


async def set_future_result(future, delay, result):
    await asyncio.sleep(delay)
    future.set_result(42)


def callback(future):
    print("future is finished! result is", future.result())


async def main():
    loop = asyncio.get_running_loop()
    future = loop.create_future()
    future.add_done_callback(callback)
    loop.create_task(set_future_result(future, 2, 42))
    await future


asyncio.run(main())  # 输出为：future is finished! result is 42
```
注意，需要安排一个任务来设置future的结果，而不能在`add_done_callback()`回调函数里面设置，因为`future.set_result()`会同时将future的状态设置为`done`，而`add_done_callback`只有在future的状态为done时才会触发，因此如果在`add_done_callback()`设置future的结果，`await future`会导致程序挂死。

### run_in_executor

`run_in_executor`可以新开一个线程或者一个进程来运行同步的函数，返回一个asyncio的future，这样就可以通过协程来并发的运行多个同步函数。比如有两个同步函数，一个是io阻塞型的，一个是计算密集型的，一般情况下，我们只能够同步的运行这两个函数，比如：
```python
def blocking_io():
    print(time.strftime("%X"), "start blocking io")
    time.sleep(3)
    return 42


def cpu_bound():
    print(time.strftime("%X"), "start cpu bound")
    return sum(i * i for i in range(10 ** 7))

blocking_io_result = blocking_io()
cpu_bound_result = cpu_bound()
print(blocking_io_result, cpu_bound_result)
```
但是，现在我们为了提高性能，想要同时运行这两个函数，传统的做法比较麻烦（可以开两个线程，其中一个线程对cpu_bound做封装，在内部使用进程来运行cpu_bound），如果利用asyncio，则十分简单：
```python
import asyncio
import concurrent.futures
import time


def blocking_io():
    print(time.strftime("%X"), "start blocking io")
    time.sleep(2)
    return 42


def cpu_bound():
    print(time.strftime("%X"), "start cpu bound")
    return sum(i * i for i in range(10 ** 7))


async def async_blocking_io():
    loop = asyncio.get_running_loop()
    with concurrent.futures.ProcessPoolExecutor() as pool:
        result = await loop.run_in_executor(pool, cpu_bound)
        print(time.strftime("%X"), "blocking io result is", result)


async def async_cpu_bound():
    loop = asyncio.get_running_loop()
    with concurrent.futures.ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(pool, blocking_io)
        print(time.strftime("%X"), "cpu bound result is", result)


async def main():
    await asyncio.gather(async_cpu_bound(), async_blocking_io())


if __name__ == "__main__":
    asyncio.run(main())
```
输出结果如下：
```
16:53:25 start blocking io
16:53:25 start cpu bound
16:53:26 blocking io result is 333333283333335000000
16:53:27 cpu bound result is 42
```
这里有一点要非常注意，就是创建新的进程会复制整个代码，所以调用的语句`asyncio.run(main())`必须放在`if __name__ == "__main__":`中，否则会抛出错误，而且抛出的错误说明并不直观，具体原因见《python那些事儿》笔记的multiprocessing模块一节。

### asyncio.Future vs concurrent.futures.Future

`asyncio.Future`虽然和`concurrent.futures.Future`虽然有十分类似的API，但是两者其实是完全不同的。但是`concurrent.futures.Future`可以通过`asyncio.wrap_future`转换为`asyncio.Future`，其实在`run_in_executor`内部就是这么做的，也可以手动进行转换：
```python
import asyncio
import random
from concurrent.futures import ThreadPoolExecutor
from time import sleep


def return_after_5_secs(message):
    sleep(5)
    return message


pool = ThreadPoolExecutor(3)


async def doit():
    identify = random.randint(1, 100)
    future = pool.submit(return_after_5_secs, (f"result: {identify}"))
    awaitable = asyncio.wrap_future(future)
    print(f"waiting result: {identify}")
    return await awaitable


async def app():
    # run some stuff multiple times
    tasks = [doit(), doit()]

    result = await asyncio.gather(*tasks)
    print(result)

print("waiting app")
asyncio.run(app())
```
输出为：
```
waiting app
waiting result: 76
waiting result: 87
['result: 76', 'result: 87']
```